In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

  Activating project at `~/Documents/eth_courses/notebooks/control/MPC`


In [112]:
using PyCall
import Random
using Convex
using LinearAlgebra

# SOlvers,
import ECOS, OSQP

cvx = pyimport("cvxpy");
cvxpygen = pyimport("cvxpygen");
cpg = pyimport("cvxpygen.cpg");
np = pyimport("numpy");

# operator overloading for matrix multiplication in cvx
Base.:*(A::Array ,B::PyObject) = cvx.matmul(A,B)

## Generate data

In [145]:
nx = 4

Q = rand(nx,nx)
Q = Q'*Q + I

Q_n = rand(nx)

# System
A = collect(I(nx))
b = [10; 10; 10; 10];

# Using Cvxpy

In [101]:

# Define and solve the cvxXPY problem.
x = cvx.Variable(nx)

# define cost
cost = (1/2) * cvx.quad_form(x, Q) + Q_n*x

# constrains
constrains_vec = []
push!(constrains_vec, A * x<= b)
push!(constrains_vec, x >= 1)
push!(constrains_vec, x <= 10)

prob = cvx.Problem(cvx.Minimize(cost),constrains_vec)

prob.solve(solver=cvx.OSQP)

# Print result.
print("\nThe optimal value is: ", prob.value)


The optimal value is: 7.160964158433265

# Using Convex.jl

In [106]:
x = Variable(nx)

cost = (1/2) * Convex.quadform(x,Q)+ dot(Q_n,x)

# objective 
p = minimize(cost) # or c' * x

# constrains
p.constraints += A * x <= b;
p.constraints += [x >= 1; x <= 10];

solve!(p, ECOS.Optimizer; silent_solver = true)

println(p.optval)

7.160964149343695


# Code generation

In [201]:
let

n = 4

# System
A_val = collect(I(4))
b_val = [10; 10; 10; 10];
    
x = cvx.Variable(n, name="x")
A = cvx.Parameter((n, n), name="A")
b = cvx.Parameter(n, name="b")

# cost function
cost = cvx.sum_squares(cvx.matmul(A, x) - b)

# constrains
constrains_vec = []
push!(constrains_vec, cvx.matmul(A, x) <= b)
push!(constrains_vec, x >= 1)
push!(constrains_vec, x <= 10)

    
problem = cvx.Problem(cvx.Minimize(cost), [x >= 0])
cpg.generate_code(problem, code_dir="MPC_Code", solver="OSQP")

# A.value = np.array([1 0 0 0; 0 1 0 0 ; 0 0 1 0; 0 0 0 1])
# b.value = np.array(b_val)
end

-- The C compiler identification is AppleClang 12.0.0.12000032
-- The CXX compiler identification is AppleClang 12.0.0.12000032
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Setting build type to 'Release' as none was specified.
-- Single precision floats (32bit) are OFF
-- Long integers (64bit) are OFF
-- COMPILER_OPTS = -DUSE_LAPACK -DCTRLC
-- Configuring done (1.3s)
-- Generating done (0.1s)
-- Build files have been written to: /Users/sarathmenon/Documents/eth_courses/notebooks/control/MPC/nonneg_LS/c/build
[  0%] Building C object CMakeFiles/cpg.dir/src/

/usr/local/lib/python3.11/site-packages/setuptools/__init__.py:84: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
/Library/Developer/CommandLineTools/usr/bin/ranlib: file: out/libcpg.a(ctrlc.c.o) has no symbols
/Library/Developer/CommandLineTools/usr/bin/ranlib: file: out/libcpg.a(amd_dump.c.o) has no symbols
/Library/Developer/CommandLineTools/usr/bin/ranlib: file: out/libcpg.a(amd_global.c.o) has no symbols
/Library/Developer/CommandLineTools/usr/bin/ranlib: file: out/libcpg.a(ctrlc.c.o) has no symbols
/Library/Developer/CommandLineTools/usr/bin/ranlib: file: out/libcpg.a(amd_dump.c.o) has no

4×4 Matrix{Int64}:
 1  0  0  0
 0  1  0  0
 0  0  1  0
 0  0  0  1